In [1]:
#Import Required Libraries
import cv2
import numpy as np
import time
import playsound
from datetime import datetime

In [2]:
#Load Haar Cascades Classifier for face and eye detection
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml') 
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')

alarm = False
counter = 0
start = ''

#Detect drowsiness symptoms
def drowsiness_detector(image):
    
    #Change colored image itno grayscale image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Detect face with the help of Haar Cascades Classifier
    faces = face_classifier.detectMultiScale(gray, 1.3,5)
    
    #If no face detected return the original image
    if faces is ():
        return image
    
    
    for (x,y,w,h) in faces:
        #Highlighting faces if present
        cv2.rectangle(image,(x,y), (x+w, y+h),(0,0,255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        
        #Eye detection in case of face detected
        eyes = eye_classifier.detectMultiScale(roi_gray, 1.3, 4)
        
        if eyes is ():
            global counter, start, end
            counter +=1
            if counter == 1:
                start = datetime.now()
            end = datetime.now()
            #If no eye detected for more than 10 sec, alarm starts.
            if (end - start).seconds == 10:
                alarm = True
                print('alarm starts at', datetime.now().time() )
                playsound.playsound('alarm.wav')
            
            return image
        
        #Highlighting eyes if present
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(image, (x+ex, y+ey), (x+ex+ew,y+ey+eh), (0,255,0), 2)
            
    #Set values back to initial in case of no drowsiness detected      
    counter = 0  
    start = ''
    return image

In [3]:
#Initialize the web camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('Drowsiness Detector', drowsiness_detector(frame))
    if cv2.waitKey(1)==13: #Press Enter Key to stop
        break
        
cap.release()
cv2.destroyAllWindows()

alarm starts at 20:05:41.595580
